In [1]:
import requests
from bs4 import BeautifulSoup
import re
import datetime

response = requests.get('https://movie.naver.com/movie/sdb/rank/rmovie.naver')
soup = BeautifulSoup(response.content, 'html.parser')

top10_lst = soup.select("td.title > div > a")[:10]
top10_title = list()
top10_href = list()

# title, href 추출
for top10 in top10_lst:
    title = top10.get_text()
    href = 'https://movie.naver.com' + str(top10).split("\"")[1]
    top10_title.append(title)
    top10_href.append(href)

top10_score = list()
top10_overview = list()
top10_director = list()
top10_rating = list()
top10_count = list()
for href in top10_href:
    response_info = requests.get(href)
    soup_info = BeautifulSoup(response_info.content, 'html.parser')
    
    # 평점 추출
    result = list()
    try: 
        score_box = soup_info.select_one('div.main_score')
        score_1 = score_box.select_one('div.score').select('div.star_score > em')
        score_2 = score_box.select('div.spc_score_area > a > div.star_score > em')
        score_3 = score_box.select('div.score > div > a > em')
        
        result.append(score_1)
        result.append(score_2)
        result.append(score_3)
        
        score_lst = list()
        for score0 in result:
            score_str = ''
            if score0:
                for score in score0:
                    score_str += score.get_text()
                score_lst.append(float(score_str))
            else:
                score_lst.append('-')        
    except:
        score_lst = ['-', '-', '-']

    top10_score.append(score_lst)
    
    # 개요 추출
    ov_data = soup_info.select_one('dl.info_spec').select('span')
    
    ov_result = list()
    for ov in ov_data:
        ov_result.append(re.sub('\s+', '', ov.get_text()))

    top10_overview.append(ov_result[:4])
    
    # 감독
    dir_data = soup_info.select('dl.info_spec > dd')[1].select_one('p > a')
    top10_director.append(dir_data.get_text())

    # 등급 추출
    rating_data = soup_info.select('dl.info_spec > dd')[0].select('span')[2]
    top10_rating.append(rating_data.get_text())

    # 흥행 추출
    try:
      count_data = soup_info.select('dl.info_spec > dd')[4].select_one('p.count').get_text()
    except:
      if datetime.datetime.strptime(ov_result[3][:-2], '%Y.%m.%d') > datetime.datetime.today():
        count_data = '개봉 예정' 
      else:
        count_data = '내용 없음'
    top10_count.append(count_data)

print(top10_title, top10_score, top10_overview, top10_director, top10_rating, top10_count,
      len(top10_title), len(top10_score), len(top10_overview), len(top10_director), len(top10_rating), len(top10_count),
      sep='\n')

['한산: 용의 출현', '비상선언', '외계+인 1부', '탑건: 매버릭', '헤어질 결심', '헌트', '미니언즈2', '토르: 러브 앤 썬더', '더 킬러: 죽어도 되는 아이', '범죄도시2']
[[8.49, 6.89, 8.74], [6.79, 6.27, 6.6], [6.88, 6.0, 7.43], [9.59, 8.44, 9.77], [8.92, 8.71, 8.68], ['-', '-', '-'], [7.84, 5.67, 7.72], [6.73, 5.14, 6.54], [8.64, '-', 7.14], [9.02, 6.56, 9.35]]
[['액션,드라마', '한국', '129분', '2022.07.27개봉'], ['드라마', '한국', '140분', '2022.08.03개봉'], ['액션,판타지,SF', '한국', '142분', '2022.07.20개봉'], ['액션', '미국', '130분', '2022.06.22개봉'], ['멜로/로맨스,드라마,서스펜스', '한국', '138분', '2022.06.29개봉'], ['액션,드라마', '한국', '125분', '2022.08.10개봉'], ['애니메이션,모험,코미디', '미국', '87분', '2022.07.20개봉'], ['액션,모험,판타지', '미국', '119분', '2022.07.06개봉'], ['액션', '한국', '95분', '2022.07.13개봉'], ['범죄,액션', '한국', '106분', '2022.05.18개봉']]
['김한민', '한재림', '최동훈', '조셉 코신스키', '박찬욱', '이정재', '카일 발다', '타이카 와이티티', '최재훈', '이상용']
['129분 ', '140분 ', '142분 ', '130분 ', '138분 ', '125분 ', '87분 ', '119분 ', '95분 ', '106분 ']
['3,692,268명(08.05 기준)', '794,768명(08.05 기준)', '1,466,379명(08.05 기준)', '7,305,843명(08.05 기준)',

In [2]:
for title, score, ov, dir, rating, count in zip(top10_title, top10_score, top10_overview, top10_director, top10_rating, top10_count):
  print([title,
         f'관람객 평점 : {score[0]} 점', f'기자·평론가 평점 : {score[1]} 점', f'네티즌 평점 : {score[2]} 점',
         ov[0], ov[1], ov[2], ov[3],
         dir, rating, count])

['한산: 용의 출현', '관람객 평점 : 8.49 점', '기자·평론가 평점 : 6.89 점', '네티즌 평점 : 8.74 점', '액션,드라마', '한국', '129분', '2022.07.27개봉', '김한민', '129분 ', '3,692,268명(08.05 기준)']
['비상선언', '관람객 평점 : 6.79 점', '기자·평론가 평점 : 6.27 점', '네티즌 평점 : 6.6 점', '드라마', '한국', '140분', '2022.08.03개봉', '한재림', '140분 ', '794,768명(08.05 기준)']
['외계+인 1부', '관람객 평점 : 6.88 점', '기자·평론가 평점 : 6.0 점', '네티즌 평점 : 7.43 점', '액션,판타지,SF', '한국', '142분', '2022.07.20개봉', '최동훈', '142분 ', '1,466,379명(08.05 기준)']
['탑건: 매버릭', '관람객 평점 : 9.59 점', '기자·평론가 평점 : 8.44 점', '네티즌 평점 : 9.77 점', '액션', '미국', '130분', '2022.06.22개봉', '조셉 코신스키', '130분 ', '7,305,843명(08.05 기준)']
['헤어질 결심', '관람객 평점 : 8.92 점', '기자·평론가 평점 : 8.71 점', '네티즌 평점 : 8.68 점', '멜로/로맨스,드라마,서스펜스', '한국', '138분', '2022.06.29개봉', '박찬욱', '138분 ', '1,741,846명(08.05 기준)']
['헌트', '관람객 평점 : - 점', '기자·평론가 평점 : - 점', '네티즌 평점 : - 점', '액션,드라마', '한국', '125분', '2022.08.10개봉', '이정재', '125분 ', '개봉 예정']
['미니언즈2', '관람객 평점 : 7.84 점', '기자·평론가 평점 : 5.67 점', '네티즌 평점 : 7.72 점', '애니메이션,모험,코미디', '미국', '87분', '2022.07.20개봉',

In [3]:
import pandas as pd

file = pd.DataFrame()

file['제목'] = top10_title

aud_lst = list()
jour_lst = list()
net_lst = list()
for score in top10_score:
   aud_lst.append(score[0])
   jour_lst.append(score[1])
   net_lst.append(score[2])
file['관람객 평점'] = aud_lst
file['기자·평론가 평점'] = jour_lst
file['네티즌 평점'] = net_lst

genre_lst = list()
coun_lst = list()
run_time_lst = list()
open_lst = list()
for ov in top10_overview:
  genre_lst.append(ov[0])
  coun_lst.append(ov[1])
  run_time_lst.append(ov[2])
  open_lst.append(ov[3])
file['장르'] = genre_lst
file['국가'] = coun_lst
file['상영 시간'] = run_time_lst
file['개봉 일자'] = open_lst

file['감독'] = top10_director
file['상영 등급'] = top10_rating
file['관객 수'] = top10_count

file

,제목,관람객 평점,기자·평론가 평점,네티즌 평점,장르,국가,상영 시간,개봉 일자,감독,상영 등급,관객 수
0,한산: 용의 출현,8.49,6.89,8.74,"액션,드라마",한국,129분,2022.07.27개봉,김한민,129분,"3,692,268명(08.05 기준)"
1,비상선언,6.79,6.27,6.6,드라마,한국,140분,2022.08.03개봉,한재림,140분,"794,768명(08.05 기준)"
2,외계+인 1부,6.88,6.0,7.43,"액션,판타지,SF",한국,142분,2022.07.20개봉,최동훈,142분,"1,466,379명(08.05 기준)"
3,탑건: 매버릭,9.59,8.44,9.77,액션,미국,130분,2022.06.22개봉,조셉 코신스키,130분,"7,305,843명(08.05 기준)"
4,헤어질 결심,8.92,8.71,8.68,"멜로/로맨스,드라마,서스펜스",한국,138분,2022.06.29개봉,박찬욱,138분,"1,741,846명(08.05 기준)"
5,헌트,-,-,-,"액션,드라마",한국,125분,2022.08.10개봉,이정재,125분,개봉 예정
6,미니언즈2,7.84,5.67,7.72,"애니메이션,모험,코미디",미국,87분,2022.07.20개봉,카일 발다,87분,"1,837,975명(08.05 기준)"
7,토르: 러브 앤 썬더,6.73,5.14,6.54,"액션,모험,판타지",미국,119분,2022.07.06개봉,타이카 와이티티,119분,"2,713,431명(08.05 기준)"
8,더 킬러: 죽어도 되는 아이,8.64,-,7.14,액션,한국,95분,2022.07.13개봉,최재훈,95분,내용 없음
9,범죄도시2,9.02,6.56,9.35,"범죄,액션",한국,106분,2022.05.18개봉,이상용,106분,"12,692,052명(08.05 기준)"


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
file.to_csv('drive/MyDrive/crawling_hw_김재도.csv', index=False)